In [ ]:
!pip freeze
!pip3 install numpy
!pip3 install pandas
!pip3 install sklearn
!pip3 install matplotlib
!pip3 install pydotplus
!pip3 install six

Problem 2: Decisions, decisions

Implement ID3 from scratch

In [1]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/f1nn3g4n/SENG474/main/A1/elections_clean.csv?token=GHSAT0AAAAAABY5KIWHHYYRHKHNRR5SWL3IYZTRIKA', sep='\t')
df.head()

,Unnamed: 0,votes,unemployment_rate,med_hhinc,per_capita_inc,poverty_all_ages_pct,deep_pov_all,deep_pov_children,population,area_in_square_miles,...,democrat,county,state,education,religion,age_old,age_young,age_adult,ethnic_male,ethnic_female
0,0,-0.372653,0.025,0.015025,0.633407,0.160,0.034402,0.012097,-0.309896,0.224322,...,0,Sioux County,NE,GTE Bachelor's,MAINLINE CHRISTIAN,0.309524,0.266667,0.318254,WHITE MALE,WHITE FEMALE
1,1,1.587511,0.077,-0.432670,0.176372,0.233,0.120282,0.192827,1.669577,-0.291546,...,1,Baltimore city,MD,Some College,MAINLINE CHRISTIAN,0.184608,0.322959,0.435202,BLACK MALE,BLACK FEMALE
2,2,0.347824,0.027,1.221027,1.722438,0.110,0.056108,0.050497,0.185999,-0.153858,...,1,Chittenden County,VT,High School,MAINLINE CHRISTIAN,0.200555,0.342702,0.385663,WHITE MALE,WHITE FEMALE
3,3,-0.309012,0.125,-0.321118,0.119396,0.174,0.063867,0.100785,-0.296326,1.690864,...,0,Prince of Wales-Hyder Census Area,AK,GTE Bachelor's,OTHER MISC,0.196938,0.319734,0.390365,WHITE MALE,WHITE FEMALE
4,4,0.904342,0.039,1.605427,2.295003,0.082,0.042655,0.052549,0.292996,-0.101047,...,0,St. Johns County,FL,High School,MAINLINE CHRISTIAN,0.232126,0.286295,0.377837,WHITE MALE,WHITE FEMALE


In [ ]:
# https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f